In [29]:
import sqlite3

original_conn = sqlite3.connect('netflix.sqlite')
original_cursor = original_conn.cursor()
new_conn = sqlite3.connect('task1.sqlite')
new_cursor = new_conn.cursor()

new_cursor.execute('''
    CREATE TABLE IF NOT EXISTS actors (
        actor_id INTEGER PRIMARY KEY,
        actor_name TEXT UNIQUE
    )
''')
new_cursor.execute('''
    CREATE TABLE IF NOT EXISTS movies (
        movie_id INTEGER PRIMARY KEY,
        title TEXT UNIQUE
    )
''')
new_cursor.execute('''
    CREATE TABLE IF NOT EXISTS actor_movie (
        actor_id INTEGER,
        movie_id INTEGER,
        FOREIGN KEY (actor_id) REFERENCES actors (actor_id),
        FOREIGN KEY (movie_id) REFERENCES movies (movie_id),
        PRIMARY KEY (actor_id, movie_id)
    )
''')
for row in original_cursor.execute('SELECT * FROM netflix_titles'):
    title = row[2]
    cast = row[4]
    movie_id = new_cursor.execute('INSERT OR IGNORE INTO movies (title) VALUES (?)', (title,)).lastrowid
    actors = cast.split(', ')
    for actor in actors:
        actor_id = new_cursor.execute('INSERT OR IGNORE INTO actors (actor_name) VALUES (?)', (actor.strip(),)).lastrowid
        new_cursor.execute('INSERT OR IGNORE INTO actor_movie (actor_id, movie_id) VALUES (?, ?)', (actor_id, movie_id))

new_conn.commit()
result = new_cursor.execute('''
    SELECT a1.actor_name, a2.actor_name, COUNT(*) as movies_together
    FROM actor_movie am1
    JOIN actor_movie am2 ON am1.movie_id = am2.movie_id AND am1.actor_id < am2.actor_id
    JOIN actors a1 ON am1.actor_id = a1.actor_id
    JOIN actors a2 ON am2.actor_id = a2.actor_id
    GROUP BY a1.actor_name, a2.actor_name
    ORDER BY movies_together DESC
    LIMIT 1
''').fetchone()
print(result[0], result[1])
original_conn.close()
new_conn.close()


4Minute B1A4
